In [2]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go
import plotly.express  as px

In [7]:
#DEMO 全場用電量data
site = pd.read_csv(r'datasets\ETT-data\TC3 POWER 15MIN.csv')
elc = pd.read_csv(r'datasets\ETT-data\TC3 TPC 15MIN.csv')
data = pd.merge(elc,site,how='inner',on='Dtime')
col = ['Dtime'] + [f'd{c}' for c in range(2,37)] + ['Power']
data = data[col]
data = data[data.Dtime > '2022-07'].reset_index(drop=True)
data = data.drop(data.std()[(data.std() == 0)].index, axis=1)
data.to_csv(r'datasets\ETT-data\TrainOneDay.csv',index=False)

In [169]:
#訓練使用csv 路徑為.\datasets\ETT-data\TrainOneDay.csv
#欄位 '日期' ,'(features1,2,3,4)' , '預測目標'

'''
df_raw.columns: ['date', ...(other features), target feature]
'''

data = pd.read_csv(r'datasets\ETT-data\TrainOneDay.csv')

#丟棄NA與std=0欄位
data = data.dropna(axis=0).drop(data.std()[(data.std() == 0)].index, axis=1)

#調整欄位名稱OT為目標欄位
data = data.rename(columns={
    data.columns[0]:'date',  data.columns[-1]:'OT'
})
data  = data[['date'] + list(data.columns[2:-1]) + ['OT']]

#儲存處裡完成csv
data.to_csv(r'datasets\ETT-data\ETTh1.csv',index=False)

C:\Users\ZME\AppData\Local\Temp\ipykernel_628\838334311.py:11: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [170]:
#0.8訓練 0.1驗證 0.1測試
n=96
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.date,y=data.OT,line=dict(width=2)))
fig.add_vrect(
    x0=data.date.iloc[int(data.shape[0]*0.8-n)],
    x1=data.date.iloc[int(data.shape[0]*0.9-n)],
    fillcolor="red",
    opacity=0.2,
    line_width=0,
)

fig.add_vrect(
    x0=data.date.iloc[int(data.shape[0]*0.9)-n],
    x1=data.date.iloc[-1-n],
    fillcolor="green",
    opacity=0.2,
    line_width=0,
)

In [171]:

# 重要參數
# --features     type=str,  default='MS'     特徵選取 S:單變量預測單變量(y->y) MS:多變量預測單變量(x+y->y) M:多變量預測多變量(x+y->x+y)(未測試)
# --seq_len      type=int,  default=480      用多少筆資料進行預測 EX: 15min*192 = 2day 
# --label_len    type=int,  default=96       預測標籤長度 EX: 15min*96 = 1day 
# --pred_len     type=int,  default=96       預測長度    EX: 15min*96 = 1day 
# --model'       type=str,  default='SCINet' 模型紀錄名稱
# --model_name   type=str,  default='SCINet' Tensor board 上模型名稱 (啟動方式tensorboard --logdir .\event) http://localhost:6006/

# 輔助工具
# --evaluate     type=bool, default=False    進行Test data驗證(倒數10%) 結果儲存至 .\exp\ett_results
# --evaluateALL  type=bool, default=False    進行ALL data驗證 結果儲存至 .\exp\ett_results
# --infer        type=bool, default=False    落地實際預測 取.\dataset\infer.csv 倒數seq_len筆 進行預測

# 模型配置
# --train_epochs type=int,  default=100      訓練迭代次數
# --patience     type=int,  default=15       Valid set 幾次 loss 沒有下降提早終止訓練
# --hidden-size  type=int,  default=2        隱藏層數量 建議1~5
# --batch_size   type=int,  default=8        訓練批次大小 建議2,4,8,16,32,64,128,256
# --lr'          type=float default=3e-3     學習速率建議 1e-3 ~ 1e-5

In [172]:
!python .\run_ETTh.py --hidden-size 2 --batch_size 16  --seq_len 192 --label_len 96 --pred_len 96 --features MS --model_name h2_b16_2day_ms 

Args in experiment:
Namespace(model='SCINet', data='ETTh1', root_path='./datasets/ETT-data/', data_path='ETTh1.csv', features='MS', target='OT', freq='h', checkpoints='exp/ETT_checkpoints/', inverse=False, embed='timeF', use_gpu=True, gpu=0, use_multi_gpu=False, devices='0', seq_len=192, label_len=96, pred_len=96, concat_len=0, single_step=0, single_step_output_One=0, lastWeight=1.0, cols=None, num_workers=0, itr=0, train_epochs=100, batch_size=16, patience=15, lr=0.003, loss='rmse', lradj=1, use_amp=False, save=True, model_name='h2_b16_2day_ms', resume=False, evaluate=False, evaluateAll=False, infer=False, hidden_size=2.0, INN=1, kernel=5, dilation=1, dropout=0.5, positionalEcoding=False, groups=1, levels=3, stacks=1, num_decoder_layer=1, RIN=False, decompose=False, enc_in=34, dec_in=34, c_out=1, detail_freq='h')
SCINet(
  (blocks1): EncoderTree(
    (SCINet_Tree): SCINet_Tree(
      (workingblock): LevelSCINet(
        (interact): InteractorLevel(
          (level): Interactor(
     


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:02<01:53,  2.90s/it]

  8%|▊         | 3/40 [00:03<00:29,  1.25it/s]

 12%|█▎        | 5/40 [00:03<00:14,  2.39it/s]

 18%|█▊        | 7/40 [00:03<00:08,  3.74it/s]

 22%|██▎       | 9/40 [00:03<00:05,  5.29it/s]

 28%|██▊       | 11/40 [00:03<00:04,  6.95it/s]

 32%|███▎      | 13/40 [00:03<00:03,  8.50it/s]

 38%|███▊      | 15/40 [00:03<00:02, 10.18it/s]

 42%|████▎     | 17/40 [00:03<00:01, 11.66it/s]

 48%|████▊     | 19/40 [00:03<00:01, 13.02it/s]

 52%|█████▎    | 21/40 [00:04<00:01, 14.15it/s]

 57%|█████▊    | 23/40 [00:04<00:01, 14.87it/s]

 62%|██████▎   | 25/40 [00:04<00:00, 15.56it/s]

 68%|██████▊   | 27/40 [00:04<00:00, 16.15it/s]

 72%|███████▎  | 29/40 [00:04<00:00, 16.34it/s]

 78%|███████▊  | 31/40 [00:04<00:00, 16.77it/s]

 82%|████████▎ | 33/40 [00:04<00:00, 17.04it/s]

 88%|████████▊ | 35/40 [00:04<00:00, 17.18it/s]

 92%|█████████▎| 37/40 [00:05<00:00, 17.2


                  (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                  (5): Tanh()
                )
                (psi): Sequential(
                  (0): ReplicationPad1d((3, 3))
                  (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                  (2): LeakyReLU(negative_slope=0.01, inplace=True)
                  (3): Dropout(p=0.5, inplace=False)
                  (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                  (5): Tanh()
                )
                (P): Sequential(
                  (0): ReplicationPad1d((3, 3))
                  (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                  (2): LeakyReLU(negative_slope=0.01, inplace=True)
                  (3): Dropout(p=0.5, inplace=False)
                  (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                  (5): Tanh()
                )
                (U): Sequential(
                  (0): ReplicationPad1d((3, 3))
                  (1): 


 15%|█▌        | 6/40 [00:00<00:01, 17.04it/s]

 20%|██        | 8/40 [00:00<00:01, 17.29it/s]

 25%|██▌       | 10/40 [00:00<00:01, 17.44it/s]

 30%|███       | 12/40 [00:00<00:01, 17.53it/s]

 35%|███▌      | 14/40 [00:00<00:01, 17.63it/s]

 40%|████      | 16/40 [00:00<00:01, 17.70it/s]

 45%|████▌     | 18/40 [00:01<00:01, 17.70it/s]

 50%|█████     | 20/40 [00:01<00:01, 17.60it/s]

 55%|█████▌    | 22/40 [00:01<00:01, 17.35it/s]

 60%|██████    | 24/40 [00:01<00:00, 17.50it/s]

 65%|██████▌   | 26/40 [00:01<00:00, 17.56it/s]

 70%|███████   | 28/40 [00:01<00:00, 17.65it/s]

 75%|███████▌  | 30/40 [00:01<00:00, 17.66it/s]

 80%|████████  | 32/40 [00:01<00:00, 17.72it/s]

 85%|████████▌ | 34/40 [00:01<00:00, 17.72it/s]

 90%|█████████ | 36/40 [00:02<00:00, 17.76it/s]

 95%|█████████▌| 38/40 [00:02<00:00, 17.74it/s]

100%|██████████| 40/40 [00:02<00:00, 17.57it/s]


100%|██████████| 1/1 [00:00<00:00, 58.83it/s]


100%|██████████| 1/1 [00:00<00:00, 47.61it/s]

  3%|▎         | 3/100 

In [173]:
!python .\run_ETTh.py --hidden-size 2 --batch_size 16  --seq_len 192 --label_len 96 --pred_len 96 --features MS --model_name h2_b16_2day_ms --evaluateAll True


                  (0): ReplicationPad1d((3, 3))
                  (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                  (2): LeakyReLU(negative_slope=0.01, inplace=True)
                  (3): Dropout(p=0.5, inplace=False)
                  (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                  (5): Tanh()
                )
                (psi): Sequential(
                  (0): ReplicationPad1d((3, 3))
                  (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                  (2): LeakyReLU(negative_slope=0.01, inplace=True)
                  (3): Dropout(p=0.5, inplace=False)
                  (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                  (5): Tanh()
                )
                (P): Sequential(
                  (0): ReplicationPad1d((3, 3))
                  (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                  (2): LeakyReLU(negative_slope=0.01, inplace=True)
                  (3): Dropout(p=0.5


 75%|███████▌  | 30/40 [00:01<00:00, 17.58it/s]

 80%|████████  | 32/40 [00:01<00:00, 17.61it/s]

 85%|████████▌ | 34/40 [00:01<00:00, 17.64it/s]

 90%|█████████ | 36/40 [00:02<00:00, 17.61it/s]

 95%|█████████▌| 38/40 [00:02<00:00, 17.64it/s]

100%|██████████| 40/40 [00:02<00:00, 17.47it/s]


100%|██████████| 1/1 [00:00<00:00, 55.56it/s]


100%|██████████| 1/1 [00:00<00:00, 47.62it/s]

  5%|▌         | 5/100 [00:14<04:05,  2.59s/it]

  0%|          | 0/40 [00:00<?, ?it/s]

  5%|▌         | 2/40 [00:00<00:02, 16.95it/s]

 10%|█         | 4/40 [00:00<00:02, 16.75it/s]

 15%|█▌        | 6/40 [00:00<00:01, 17.17it/s]

 20%|██        | 8/40 [00:00<00:01, 17.38it/s]

 25%|██▌       | 10/40 [00:00<00:01, 17.49it/s]

 30%|███       | 12/40 [00:00<00:01, 17.56it/s]

 35%|███▌      | 14/40 [00:00<00:01, 17.61it/s]

 40%|████      | 16/40 [00:00<00:01, 17.59it/s]

 45%|████▌     | 18/40 [00:01<00:01, 17.62it/s]

 50%|█████     | 20/40 [00:01<00:01, 17.65it/s]

 55%|█████▌    | 22/40 [00:01<00:0

Args in experiment:
Namespace(model='SCINet', data='ETTh1', root_path='./datasets/ETT-data/', data_path='ETTh1.csv', features='MS', target='OT', freq='h', checkpoints='exp/ETT_checkpoints/', inverse=False, embed='timeF', use_gpu=True, gpu=0, use_multi_gpu=False, devices='0', seq_len=192, label_len=96, pred_len=96, concat_len=0, single_step=0, single_step_output_One=0, lastWeight=1.0, cols=None, num_workers=0, itr=0, train_epochs=100, batch_size=16, patience=15, lr=0.003, loss='rmse', lradj=1, use_amp=False, save=True, model_name='h2_b16_2day_ms', resume=False, evaluate=False, evaluateAll=True, infer=False, hidden_size=2.0, INN=1, kernel=5, dilation=1, dropout=0.5, positionalEcoding=False, groups=1, levels=3, stacks=1, num_decoder_layer=1, RIN=False, decompose=False, enc_in=34, dec_in=34, c_out=1, detail_freq='h')
SCINet(
  (blocks1): EncoderTree(
    (SCINet_Tree): SCINet_Tree(
      (workingblock): LevelSCINet(
        (interact): InteractorLevel(
          (level): Interactor(
      


 22%|██▏       | 193/891 [00:04<00:11, 58.47it/s]


        )
      )
      (SCINet_Tree_odd): SCINet_Tree(
        (workingblock): LevelSCINet(
          (interact): InteractorLevel(
            (level): Interactor(
              (split): Splitting()
              (phi): Sequential(
                (0): ReplicationPad1d((3, 3))
                (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                (2): LeakyReLU(negative_slope=0.01, inplace=True)
                (3): Dropout(p=0.5, inplace=False)
                (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                (5): Tanh()
              )
              (psi): Sequential(
                (0): ReplicationPad1d((3, 3))
                (1): Conv1d(34, 68, kernel_size=(5,), stride=(1,))
                (2): LeakyReLU(negative_slope=0.01, inplace=True)
                (3): Dropout(p=0.5, inplace=False)
                (4): Conv1d(68, 34, kernel_size=(3,), stride=(1,))
                (5): Tanh()
              )
              (P): Sequential(
                (0):


 92%|█████████▏| 817/891 [00:15<00:01, 58.97it/s]


In [27]:
from ipywidgets import interact
from glob import glob
import matplotlib.pyplot as plt
files = glob(r'exp\ett_results\*')

from sklearn.metrics import mean_squared_error
def rmse(true,pred):
    return mean_squared_error(true.flatten(),pred.mean(axis=2).flatten())**0.5

ModuleNotFoundError: No module named 'matplotlib'

In [175]:
@interact
def load(f=files):
    global true,pred
    true = np.load(rf'{f}\true_scales.npy')
    pred = np.load(rf'{f}\pred_scales.npy')
    L = true.shape[0]
    print('TRAIN RMSE:{:.2f}'.format(rmse(true[0:int(L*0.8)],pred[0:int(L*0.8)])))
    print('VALID RMSE:{:.2f}'.format(rmse(true[int(L*0.8):int(L*0.9)],pred[int(L*0.8):int(L*0.9)])))
    print('TEST  RMSE:{:.2f}'.format(rmse(true[int(L*0.9):-1],pred[int(L*0.9):-1])))
    @interact
    def show(x=(0,true.shape[0]-1,1)):
        fig = plt.figure(figsize=(20,5))
        if x > true.shape[0]*0.8:
            plt.title('Valid')
            if x > true.shape[0]*0.9:
                plt.title('Test')
        else:
            plt.title('Train')
        plt.ylim([true.min(), true.max()])
        plt.plot(true[x].flatten())
        plt.plot(pred[x].mean(axis=1).flatten())
        plt.show()

interactive(children=(Dropdown(description='f', options=('exp\\ett_results\\SCINet_ETTh1_ftMS_sl192_ll96_pl96_…